# 1. Preparación

In [ ]:
# Instanciar y guardar el modelo
import tensorflow as tf
import os
import numpy as np
from tensorflow.keras.applications import EfficientNetB0

model = EfficientNetB0(weights='imagenet')
model.name = "efficientnetb0"

# Guardar en formato Keras
model.save("efficientnetb0_model.keras", save_format="keras")


21834768/21834768 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


# 2. Funciones Reutilizables

In [ ]:
from load_imagenet_v2_test import load_imagenet_v2_test

# EfficientNetB0 requiere imágenes de tamaño 224x224
x_test, y_test = load_imagenet_v2_test(n=1000, target_size=(224, 224))


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imagenet_v2/matched-frequency/incomplete.0M57MN_3.0.0/imagenet_v2-test.tfr…

Dataset imagenet_v2 downloaded and prepared to /root/tensorflow_datasets/imagenet_v2/matched-frequency/3.0.0. Subsequent calls will reuse this data.
✅ Dataset cargado: 1000 muestras de 224x224


In [ ]:
from exportar_tflite_variantes import exportar_tflite_variantes

# Exportar FP32, INT8 y CH
exportar_tflite_variantes(model, x_test, nombre_base="efficientnet")


Saved artifact at '/tmp/tmpjehzri_t'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1000), dtype=tf.float32, name=None)
Captures:
  134874591885072: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  134874591886608: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  134874642889424: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134874642889232: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134874642887504: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134874642890000: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134874642886736: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134874642890576: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134874637158160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134874637158352: TensorSpec(shape=(), dtype=tf.resource, name=N

/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/convert.py:854: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


✅ Exportado: efficientnet_int8.tflite
Saved artifact at '/tmp/tmp50g2i4aa'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1000), dtype=tf.float32, name=None)
Captures:
  134874591885072: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  134874591886608: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  134874642889424: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134874642889232: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134874642887504: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134874642890000: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134874642886736: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134874642890576: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134874637158160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134874637158352: TensorSp

In [ ]:
from preprocess_input_tflite import preprocess_input_tflite

x_test_ready = preprocess_input_tflite("efficientnet", x_test)


In [ ]:
from evaluate_model import evaluate_model

acc_fp32, preds_fp32 = evaluate_model("fp32", "efficientnet_fp32.tflite", x_test_ready, y_test, quantized=False)
acc_ch, preds_ch   = evaluate_model("ch",   "efficientnet_ch.tflite",   x_test_ready, y_test, quantized=False)
acc_int8, preds_i8 = evaluate_model("int8", "efficientnet_int8.tflite", x_test_ready, y_test, quantized=True)


/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


✅ Modelo: FP32  → 🎯 Accuracy: 0.6120
✅ Modelo: CH  → 🎯 Accuracy: 0.6070
✅ Modelo: INT8  → 🎯 Accuracy: 0.4590


# 3. Inspección

In [ ]:
from calcular_muestreo_estadistico import calcular_muestreo_estadistico
import inspeccionar_tensores_inyectables

# Invocación FP32
df_fp32 = inspeccionar_tensores_inyectables.inspeccionar_tensores_inyectables("efficientnet_fp32.tflite", tipo="fp32")

# ✅ Mostrar los tensores inyectables
print(f"✅ Tensores inyectables detectados: {len(df_fp32)}")
display(df_fp32)


✅ Tensores inyectables detectados: 165


,IndexModelo,Nombre,TipoDato,Forma,# Pesos,Bits por Peso,Total Bits Inyectables,n (muestra estadística)
0,1,arith.constant,<class 'numpy.float32'>,[1152],1152,32,36864,385
1,2,arith.constant1,<class 'numpy.float32'>,[1152],1152,32,36864,385
2,3,arith.constant2,<class 'numpy.float32'>,[1152],1152,32,36864,385
3,4,arith.constant3,<class 'numpy.float32'>,[1152],1152,32,36864,385
4,5,arith.constant4,<class 'numpy.float32'>,[672],672,32,21504,385
...,...,...,...,...,...,...,...,...
160,161,arith.constant160,<class 'numpy.float32'>,"[8, 1, 1, 32]",256,32,8192,385
161,162,arith.constant161,<class 'numpy.float32'>,"[32, 3, 3, 3]",864,32,27648,385
162,163,arith.constant162,<class 'numpy.float32'>,"[1000, 1280]",1280000,32,40960000,385
163,164,arith.constant163,<class 'numpy.float32'>,"[1, 1, 1, 3]",3,32,96,385


In [ ]:
# Invocación CH
df_ch = inspeccionar_tensores_inyectables.inspeccionar_tensores_inyectables("efficientnet_ch.tflite", tipo="ch")

# ✅ Mostrar los tensores inyectables
print(f"✅ Tensores inyectables detectados: {len(df_ch)}")
display(df_ch)


✅ Tensores inyectables detectados: 168


/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


,IndexModelo,Nombre,TipoDato,Forma,# Pesos,Bits por Peso,Total Bits Inyectables,n (muestra estadística)
0,1,arith.constant,<class 'numpy.float32'>,[3],3,32,96,385
1,2,arith.constant1,<class 'numpy.float32'>,[],1,32,32,385
2,14,arith.constant13,<class 'numpy.float32'>,[1000],1000,32,32000,385
3,15,arith.constant14,<class 'numpy.float32'>,"[1, 1, 1, 3]",3,32,96,385
4,16,arith.constant15,<class 'numpy.float32'>,"[1, 1, 1, 3]",3,32,96,385
...,...,...,...,...,...,...,...,...
163,175,tfl.pseudo_qconst65,<class 'numpy.int8'>,"[24, 1, 1, 144]",3456,8,27648,385
164,176,tfl.pseudo_qconst66,<class 'numpy.int8'>,"[144, 1, 1, 24]",3456,8,27648,385
165,177,tfl.pseudo_qconst67,<class 'numpy.int8'>,"[24, 1, 1, 96]",2304,8,18432,385
166,178,tfl.pseudo_qconst68,<class 'numpy.int8'>,"[96, 1, 1, 16]",1536,8,12288,385


In [ ]:
# Invocación INT8
df_int8 = inspeccionar_tensores_inyectables.inspeccionar_tensores_inyectables("efficientnet_int8.tflite", tipo="int8")

# ✅ Mostrar los tensores inyectables
print(f"✅ Tensores inyectables detectados: {len(df_int8)}")
display(df_int8)


✅ Tensores inyectables detectados: 86


/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


,IndexModelo,Nombre,TipoDato,Forma,# Pesos,Bits por Peso,Total Bits Inyectables,n (muestra estadística)
0,13,tfl.pseudo_qconst1,<class 'numpy.int8'>,"[1000, 1280]",1280000,8,10240000,385
1,15,tfl.pseudo_qconst3,<class 'numpy.int8'>,"[1280, 1, 1, 320]",409600,8,3276800,385
2,17,tfl.pseudo_qconst5,<class 'numpy.int8'>,"[320, 1, 1, 1152]",368640,8,2949120,385
3,19,tfl.pseudo_qconst7,<class 'numpy.int8'>,"[1152, 1, 1, 48]",55296,8,442368,385
4,21,tfl.pseudo_qconst9,<class 'numpy.int8'>,"[48, 1, 1, 1152]",55296,8,442368,385
...,...,...,...,...,...,...,...,...
81,175,tfl.pseudo_qconst163,<class 'numpy.int8'>,"[32, 3, 3, 3]",864,8,6912,385
82,176,tfl.pseudo_qconst164,<class 'numpy.int8'>,[3],3,8,24,385
83,177,tfl.pseudo_qconst165,<class 'numpy.int8'>,"[1, 1, 1, 3]",3,8,24,385
84,178,tfl.pseudo_qconst166,<class 'numpy.int8'>,"[1, 1, 1, 3]",3,8,24,385


# 4. Inyección

# Invocación INT 8

In [ ]:
from campaña_sbf_modelo import campaña_sbf_modelo

# Leer el modelo INT8 .tflite en memoria
with open("efficientnet_int8.tflite", "rb") as f:
    model_int8 = f.read()

# Usar solo los primeros 5 tensores para testeo
df_int8_reducido = df_int8.head(10)

# Ejecutar campaña SBF sobre pocos tensores INT8
df_resultados_int8 = campaña_sbf_modelo(
    model_content=model_int8,
    df_tensores=df_int8,
    x_input=x_test_ready,
    y_true=y_test,
    acc_baseline=acc_int8,
    tipo_modelo="int8",
    N=1
)

# Mostrar resultados
from IPython.display import display
print("🔍 Resultados INT8 (test reducido):")
display(df_resultados_int8)

# Guardar resultados en csv
df_resultados_int8.to_csv("resultados_int8.csv", index=False)



🧬 Inyectando en: tfl.pseudo_qconst1 (index=13)


/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


✅ Modelo: INT8  → 🎯 Accuracy: 0.4590

🧬 Inyectando en: tfl.pseudo_qconst3 (index=15)
✅ Modelo: INT8  → 🎯 Accuracy: 0.4590

🧬 Inyectando en: tfl.pseudo_qconst5 (index=17)
✅ Modelo: INT8  → 🎯 Accuracy: 0.4600

🧬 Inyectando en: tfl.pseudo_qconst7 (index=19)
✅ Modelo: INT8  → 🎯 Accuracy: 0.4590

🧬 Inyectando en: tfl.pseudo_qconst9 (index=21)
✅ Modelo: INT8  → 🎯 Accuracy: 0.4590

🧬 Inyectando en: tfl.pseudo_qconst11 (index=23)
✅ Modelo: INT8  → 🎯 Accuracy: 0.4590

🧬 Inyectando en: tfl.pseudo_qconst13 (index=25)
✅ Modelo: INT8  → 🎯 Accuracy: 0.4570

🧬 Inyectando en: tfl.pseudo_qconst15 (index=27)
✅ Modelo: INT8  → 🎯 Accuracy: 0.4590

🧬 Inyectando en: tfl.pseudo_qconst17 (index=29)
✅ Modelo: INT8  → 🎯 Accuracy: 0.4590

🧬 Inyectando en: tfl.pseudo_qconst19 (index=31)
✅ Modelo: INT8  → 🎯 Accuracy: 0.4610

🧬 Inyectando en: tfl.pseudo_qconst21 (index=33)
✅ Modelo: INT8  → 🎯 Accuracy: 0.4590

🧬 Inyectando en: tfl.pseudo_qconst23 (index=35)
✅ Modelo: INT8  → 🎯 Accuracy: 0.4580

🧬 Inyectando en: tfl

,Index Modelo,Tensor,Forma,# Pesos,Total Bits Inyectables,Pesos Modificados,Acc Baseline,Acc Post-inyección,Degradación (%)
77,167,tfl.pseudo_qconst155,"(16, 1, 1, 32)",512,4096,1,45.9,32.0,-13.9
66,145,tfl.pseudo_qconst133,"(144, 1, 1, 24)",3456,27648,1,45.9,45.6,-0.3
6,25,tfl.pseudo_qconst13,"(1152, 1, 1, 192)",221184,1769472,1,45.9,45.7,-0.2
11,35,tfl.pseudo_qconst23,"(1152, 1, 1, 192)",221184,1769472,1,45.9,45.8,-0.1
15,43,tfl.pseudo_qconst31,"(1, 5, 5, 1152)",28800,230400,1,45.9,45.8,-0.1
...,...,...,...,...,...,...,...,...,...
79,171,tfl.pseudo_qconst159,"(8, 1, 1, 32)",256,2048,1,45.9,47.1,1.2
46,105,tfl.pseudo_qconst93,"(480, 1, 1, 80)",38400,307200,1,45.9,47.2,1.3
51,115,tfl.pseudo_qconst103,"(480, 1, 1, 80)",38400,307200,1,45.9,47.2,1.3
76,165,tfl.pseudo_qconst153,"(96, 1, 1, 16)",1536,12288,1,45.9,47.4,1.5


In [ ]:
from clasificar_tensores_sensibles import clasificar_tensores_sensibles

df_clasificados_int8 = clasificar_tensores_sensibles(df_resultados_int8, tipo_modelo="int8")

# Mostrar solo los tensores sensibles
df_sensibles_int8 = df_clasificados_int8[df_clasificados_int8["TENSOR SENSIBLE?"] == "SENSIBLE"]
display(df_sensibles_int8)

,Index Modelo,Tensor,Forma,# Pesos,Total Bits Inyectables,Pesos Modificados,Acc Baseline,Acc Post-inyección,Degradación (%),TENSOR SENSIBLE?
77,167,tfl.pseudo_qconst155,"(16, 1, 1, 32)",512,4096,1,45.9,32.0,-13.9,SENSIBLE


In [ ]:
# importar funcion

#  Cargar bytes del modelo
with open("efficientnet_int8.tflite", "rb") as f:
    tflite_bytes_int8 = f.read()


df_bits = bit_sweep_modelo(
    model_content=tflite_bytes_int8,
    tensor_name="tfl.pseudo_qconst155",
    x_input=x_test_ready,
    y_true=y_test,
    acc_baseline=acc_int8,
    tipo_modelo="int8",
    k_flips_por_bit=1
)
display(df_bits.head())


/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


✅ Modelo: INT8  → 🎯 Accuracy: 0.4680
✅ Modelo: INT8  → 🎯 Accuracy: 0.4550
✅ Modelo: INT8  → 🎯 Accuracy: 0.4630
✅ Modelo: INT8  → 🎯 Accuracy: 0.4700
✅ Modelo: INT8  → 🎯 Accuracy: 0.4640
✅ Modelo: INT8  → 🎯 Accuracy: 0.4700
✅ Modelo: INT8  → 🎯 Accuracy: 0.2920
✅ Modelo: INT8  → 🎯 Accuracy: 0.4670
🔎 Bit Sweep → Tensor: tfl.pseudo_qconst155
   • Acc Baseline: 45.9000%
   • Acc Post-Barrido (media): 44.3625%
   • Degradación media: -1.5375%


,Tensor,Bit Pos,Flips por Bit,Acc Baseline,Acc Post-Bit,Degradación (%),Notas
0,tfl.pseudo_qconst155,0,1,45.9,46.8,0.9,
1,tfl.pseudo_qconst155,1,1,45.9,45.5,-0.4,
2,tfl.pseudo_qconst155,2,1,45.9,46.3,0.4,
3,tfl.pseudo_qconst155,3,1,45.9,47.0,1.1,
4,tfl.pseudo_qconst155,4,1,45.9,46.4,0.5,


In [ ]:
# Importar funcion

# Leer el modelo en memoria
with open("efficientnet_int8.tflite", "rb") as f:
    model_int8 = f.read()

# df_sensibles_int8: viene de clasificar_tensores_sensibles(...) filtrado == "SENSIBLE"
df_mbf = campaña_mbf_modelo(
    model_content=model_int8,        # bytes del .tflite (no ruta)
    df_tensores=df_sensibles_int8,   # solo tensores sensibles
    x_input=x_test_ready,        # tu subconjunto (~3k imgs) ya preprocesado
    y_true=y_test,               # labels del subconjunto
    acc_baseline=acc_int8,       # baseline del mismo subconjunto
    tipo_modelo="int8",              # "int8" | "fp32" | "ch"
    progresion=[1,25,50,75,100],     # como pediste
    random_seed=0                    # opcional para reproducibilidad
)

display(df_mbf)



🧪 MBF en tensor: tfl.pseudo_qconst155


/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


✅ Modelo: INT8  → 🎯 Accuracy: 0.4670
✅ Modelo: INT8  → 🎯 Accuracy: 0.4700
✅ Modelo: INT8  → 🎯 Accuracy: 0.3690
✅ Modelo: INT8  → 🎯 Accuracy: 0.0290
✅ Modelo: INT8  → 🎯 Accuracy: 0.1580
  ✅ MBF completado en tfl.pseudo_qconst155.


,Index Modelo,Tensor,Forma,# Pesos,Total Bits Inyectables,K (flips simultáneos),Acc Baseline,Acc Post-inyección,Degradación (%),Notas
0,167,tfl.pseudo_qconst155,"(16, 1, 1, 32)",512,4096,1,45.9,46.7,0.8,
1,167,tfl.pseudo_qconst155,"(16, 1, 1, 32)",512,4096,25,45.9,47.0,1.1,
2,167,tfl.pseudo_qconst155,"(16, 1, 1, 32)",512,4096,50,45.9,36.9,-9.0,
3,167,tfl.pseudo_qconst155,"(16, 1, 1, 32)",512,4096,75,45.9,2.9,-43.0,
4,167,tfl.pseudo_qconst155,"(16, 1, 1, 32)",512,4096,100,45.9,15.8,-30.1,


# Invocación CH

In [ ]:
from campaña_sbf_modelo import campaña_sbf_modelo

# Leer el modelo CH .tflite en memoria
with open("efficientnet_ch.tflite", "rb") as f:
    model_ch = f.read()

# Usar solo los primeros 5 tensores para testeo
df_ch_reducido = df_ch.head(10)

# Ejecutar campaña SBF sobre pocos tensores CH
df_resultados_ch = campaña_sbf_modelo(
    model_content=model_ch,
    df_tensores=df_ch_reducido,
    x_input=x_test_ready,
    y_true=y_test,
    acc_baseline=acc_ch,
    tipo_modelo="ch",
    N=100
)

# Mostrar resultados
from IPython.display import display
print("🔍 Resultados CH (test reducido):")
display(df_resultados_ch)

# Guardar resultados en csv
df_resultados_ch.to_csv("resultados_ch.csv", index=False)



🧬 Inyectando en: arith.constant (index=1)


/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


✅ Modelo: CH  → 🎯 Accuracy: 0.0010

🧬 Inyectando en: arith.constant1 (index=2)
✅ Modelo: CH  → 🎯 Accuracy: 0.0000

🧬 Inyectando en: arith.constant13 (index=14)
✅ Modelo: CH  → 🎯 Accuracy: 0.0000

🧬 Inyectando en: arith.constant14 (index=15)
✅ Modelo: CH  → 🎯 Accuracy: 0.0010

🧬 Inyectando en: arith.constant15 (index=16)
✅ Modelo: CH  → 🎯 Accuracy: 0.0020

🧬 Inyectando en: arith.constant16 (index=17)
✅ Modelo: CH  → 🎯 Accuracy: 0.0010

🧬 Inyectando en: arith.constant17 (index=18)
✅ Modelo: CH  → 🎯 Accuracy: 0.0010

🧬 Inyectando en: arith.constant18 (index=19)
✅ Modelo: CH  → 🎯 Accuracy: 0.5230

🧬 Inyectando en: arith.constant19 (index=20)
✅ Modelo: CH  → 🎯 Accuracy: 0.0010

🧬 Inyectando en: arith.constant20 (index=21)
✅ Modelo: CH  → 🎯 Accuracy: 0.6090
🔍 Resultados CH (test reducido):


,Index Modelo,Tensor,Forma,# Pesos,Total Bits Inyectables,Pesos Modificados,Acc Baseline,Acc Post-inyección,Degradación (%)
1,2,arith.constant1,(),1,32,4,60.7,0.0,-60.7
2,14,arith.constant13,"(1000,)",1000,32000,98,60.7,0.0,-60.7
0,1,arith.constant,"(3,)",3,96,12,60.7,0.1,-60.6
3,15,arith.constant14,"(1, 1, 1, 3)",3,96,12,60.7,0.1,-60.6
5,17,arith.constant16,"(32, 3, 3, 3)",864,27648,98,60.7,0.1,-60.6
6,18,arith.constant17,"(8, 1, 1, 32)",256,8192,99,60.7,0.1,-60.6
8,20,arith.constant19,"(16, 1, 1, 32)",512,16384,99,60.7,0.1,-60.6
4,16,arith.constant15,"(1, 1, 1, 3)",3,96,12,60.7,0.2,-60.5
7,19,arith.constant18,"(32, 1, 1, 8)",256,8192,99,60.7,52.3,-8.4
9,21,arith.constant20,"(4, 1, 1, 96)",384,12288,93,60.7,60.9,0.2


In [ ]:
# Clasificar tensores sensibles
from clasificar_tensores_sensibles import clasificar_tensores_sensibles

df_clasificados_ch = clasificar_tensores_sensibles(df_resultados_ch, tipo_modelo="ch")

# Mostrar solo los tensores sensibles
df_sensibles_ch = df_clasificados_ch[df_clasificados_ch["TENSOR SENSIBLE?"] == "SENSIBLE"]
display(df_sensibles_ch)

,Index Modelo,Tensor,Forma,# Pesos,Total Bits Inyectables,Pesos Modificados,Acc Baseline,Acc Post-inyección,Degradación (%),TENSOR SENSIBLE?
1,2,arith.constant1,(),1,32,4,60.7,0.0,-60.7,SENSIBLE
2,14,arith.constant13,"(1000,)",1000,32000,98,60.7,0.0,-60.7,SENSIBLE
0,1,arith.constant,"(3,)",3,96,12,60.7,0.1,-60.6,SENSIBLE
3,15,arith.constant14,"(1, 1, 1, 3)",3,96,12,60.7,0.1,-60.6,SENSIBLE
5,17,arith.constant16,"(32, 3, 3, 3)",864,27648,98,60.7,0.1,-60.6,SENSIBLE
6,18,arith.constant17,"(8, 1, 1, 32)",256,8192,99,60.7,0.1,-60.6,SENSIBLE
8,20,arith.constant19,"(16, 1, 1, 32)",512,16384,99,60.7,0.1,-60.6,SENSIBLE
4,16,arith.constant15,"(1, 1, 1, 3)",3,96,12,60.7,0.2,-60.5,SENSIBLE


In [ ]:
# Cargar bytes del modelo
with open("efficientnet_ch.tflite", "rb") as f:
    tflite_bytes_ch = f.read()
# Invocar la funcion de barrido
df_bits = bit_sweep_modelo(
    model_content=tflite_bytes_ch,
    tensor_name="arith.constant19",
    x_input=x_test_ready,
    y_true=y_test,
    acc_baseline=acc_ch,
    tipo_modelo="ch",
    k_flips_por_bit=1
)
display(df_bits.head())

/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


✅ Modelo: CH  → 🎯 Accuracy: 0.6070
✅ Modelo: CH  → 🎯 Accuracy: 0.6070
✅ Modelo: CH  → 🎯 Accuracy: 0.6080
✅ Modelo: CH  → 🎯 Accuracy: 0.6070
✅ Modelo: CH  → 🎯 Accuracy: 0.6070
✅ Modelo: CH  → 🎯 Accuracy: 0.6070
✅ Modelo: CH  → 🎯 Accuracy: 0.6060
✅ Modelo: CH  → 🎯 Accuracy: 0.6080
✅ Modelo: CH  → 🎯 Accuracy: 0.6070
✅ Modelo: CH  → 🎯 Accuracy: 0.6090
✅ Modelo: CH  → 🎯 Accuracy: 0.6070
✅ Modelo: CH  → 🎯 Accuracy: 0.6110
✅ Modelo: CH  → 🎯 Accuracy: 0.6110
✅ Modelo: CH  → 🎯 Accuracy: 0.6090
✅ Modelo: CH  → 🎯 Accuracy: 0.6110
✅ Modelo: CH  → 🎯 Accuracy: 0.6040
✅ Modelo: CH  → 🎯 Accuracy: 0.6040
✅ Modelo: CH  → 🎯 Accuracy: 0.6120
✅ Modelo: CH  → 🎯 Accuracy: 0.6100
✅ Modelo: CH  → 🎯 Accuracy: 0.6040
✅ Modelo: CH  → 🎯 Accuracy: 0.6170
✅ Modelo: CH  → 🎯 Accuracy: 0.6080
✅ Modelo: CH  → 🎯 Accuracy: 0.6070
✅ Modelo: CH  → 🎯 Accuracy: 0.6100
✅ Modelo: CH  → 🎯 Accuracy: 0.6040
✅ Modelo: CH  → 🎯 Accuracy: 0.6030
✅ Modelo: CH  → 🎯 Accuracy: 0.6080
✅ Modelo: CH  → 🎯 Accuracy: 0.6090
✅ Modelo: CH  → 🎯 Ac

,Tensor,Bit Pos,Flips por Bit,Acc Baseline,Acc Post-Bit,Degradación (%),Notas
0,arith.constant19,0,1,60.7,60.7,0.0,
1,arith.constant19,1,1,60.7,60.7,0.0,
2,arith.constant19,2,1,60.7,60.8,0.1,
3,arith.constant19,3,1,60.7,60.7,0.0,
4,arith.constant19,4,1,60.7,60.7,0.0,


# Invocación FP32

In [ ]:
from campaña_sbf_modelo import campaña_sbf_modelo

# Leer el modelo  .tflite en memoria
with open("efficientnet_fp32.tflite", "rb") as f:
    model_fp32 = f.read()

# Usar solo los primeros 10 tensores para testeo
df_fp32_reducido = df_fp32.head(10)


# Ejecutar campaña SBF sobre pocos tensores FP32
df_resultados_fp32 = campaña_sbf_modelo(
    model_content=model_fp32,
    df_tensores=df_fp32_reducido,
    x_input=x_test_ready,
    y_true=y_test,
    acc_baseline=acc_fp32,
    tipo_modelo="fp32",
    N=40
)

# Mostrar resultados
from IPython.display import display
print("🔍 Resultados FP32:")
display(df_resultados_fp32)

# Guardar resultados en csv
df_resultados_fp32.to_csv("resultados_fp32.csv", index=False)




🧬 Inyectando en: arith.constant (index=1)


/usr/local/lib/python3.11/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


✅ Modelo: FP32  → 🎯 Accuracy: 0.1440

🧬 Inyectando en: arith.constant1 (index=2)
✅ Modelo: FP32  → 🎯 Accuracy: 0.0010

🧬 Inyectando en: arith.constant2 (index=3)
✅ Modelo: FP32  → 🎯 Accuracy: 0.0010

🧬 Inyectando en: arith.constant3 (index=4)
✅ Modelo: FP32  → 🎯 Accuracy: 0.0010

🧬 Inyectando en: arith.constant4 (index=5)
✅ Modelo: FP32  → 🎯 Accuracy: 0.0030

🧬 Inyectando en: arith.constant5 (index=6)
✅ Modelo: FP32  → 🎯 Accuracy: 0.0010

🧬 Inyectando en: arith.constant6 (index=7)
✅ Modelo: FP32  → 🎯 Accuracy: 0.0010

🧬 Inyectando en: arith.constant7 (index=8)
✅ Modelo: FP32  → 🎯 Accuracy: 0.0000

🧬 Inyectando en: arith.constant8 (index=9)
✅ Modelo: FP32  → 🎯 Accuracy: 0.5770

🧬 Inyectando en: arith.constant9 (index=10)
✅ Modelo: FP32  → 🎯 Accuracy: 0.0020
🔍 Resultados FP32:


,Index Modelo,Tensor,Forma,# Pesos,Total Bits Inyectables,Pesos Modificados,Acc Baseline,Acc Post-inyección,Degradación (%)
7,8,arith.constant7,"(480,)",480,15360,39,61.2,0.0,-61.2
1,2,arith.constant1,"(1152,)",1152,36864,40,61.2,0.1,-61.1
3,4,arith.constant3,"(1152,)",1152,36864,40,61.2,0.1,-61.1
2,3,arith.constant2,"(1152,)",1152,36864,40,61.2,0.1,-61.1
6,7,arith.constant6,"(672,)",672,21504,40,61.2,0.1,-61.1
5,6,arith.constant5,"(672,)",672,21504,40,61.2,0.1,-61.1
9,10,arith.constant9,"(480,)",480,15360,40,61.2,0.2,-61.0
4,5,arith.constant4,"(672,)",672,21504,40,61.2,0.3,-60.9
0,1,arith.constant,"(1152,)",1152,36864,40,61.2,14.4,-46.8
8,9,arith.constant8,"(480,)",480,15360,40,61.2,57.7,-3.5
